# Warehouse Storage Optimization

## 1. Storage Location Assignment

### Distance Matrix and Weights

#### Distance Matrix

In [ ]:
#%pip install pandas
#%pip install numpy
#%pip install scipy
#%pip install matplotlib

import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

  Using cached scipy-1.16.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.16.2-cp312-cp312-win_amd64.whl (38.6 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.0-cp312-cp312-win_amd64.whl.metadata (113 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.4-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.60.0-cp312-cp312-win_amd64.whl (2.3 MB)
Using cached kiwisolver-1.4.9-cp312-cp312-win_a

In [ ]:
# Code to calculate distance matrix
# Creating distance matrix and weight calculations for the warehouse problem.


# Define storage coordinates for 16 locations:
# Numbering: 1..4 across top row (y=4), 5..8 next (y=3), 9..12 (y=2), 13..16 (y=1)
coords = {}
for i in range(1,17):
    col = ((i-1) % 4) + 1
    row = 4 - ((i-1) // 4)
    coords[i] = (float(col), float(row))

# Define IO coordinates as reasoned:
io_coords = {
    'IO1': (2.5, 0.5),  # bottom center (equidistant from 14 and 15)
    'IO2': (0.5, 2.5),  # left middle (equidistant from 5 and 9)
    'IO3': (2.5, 4.5)   # top center (equidistant from 2 and 3)
}

# Compute Manhattan distance matrix (IO x storage)
dist_df = pd.DataFrame(index=io_coords.keys(), columns=[f"Loc{i}" for i in range(1,17)], dtype=float)
for io, iocoord in io_coords.items():
    for loc, sc in coords.items():
        dist = abs(iocoord[0] - sc[0]) + abs(iocoord[1] - sc[1])
        dist_df.loc[io, f"Loc{loc}"] = dist



# Show results
dist_df_display = dist_df.copy()


# Present as dataframes
# To display the whole dataframe
print(dist_df.T)





       IO1  IO2  IO3
Loc1   5.0  2.0  2.0
Loc2   4.0  3.0  1.0
Loc3   4.0  4.0  1.0
Loc4   5.0  5.0  2.0
Loc5   4.0  1.0  3.0
Loc6   3.0  2.0  2.0
Loc7   3.0  3.0  2.0
Loc8   4.0  4.0  3.0
Loc9   3.0  1.0  4.0
Loc10  2.0  2.0  3.0
Loc11  2.0  3.0  3.0
Loc12  3.0  4.0  4.0
Loc13  2.0  2.0  5.0
Loc14  1.0  3.0  4.0
Loc15  1.0  4.0  4.0
Loc16  2.0  5.0  5.0


*Print the distance matrix here.*



#### Weights

In [9]:
# Code to calculate weights matrix
# Given tables: frequencies (trips) for each item-IO and costs per unit load
freq = pd.DataFrame({
    'IO1':[150,60,96,175],
    'IO2':[25,200,15,135],
    'IO3':[88,150,85,90]
}, index=['A','B','C','D'])

unit_cost = pd.DataFrame({
    'IO1':[6,7,4,15],
    'IO2':[5,3,7,8],
    'IO3':[5,6,9,12]
}, index=['A','B','C','D'])

storage_spaces = pd.Series({
    'A':3, 'B':5, 'C':2, 'D':6
})

# Compute weight per item-location as sum over IO points of (frequency * unit_cost * distance)
items = ['A','B','C','D']
weight = pd.DataFrame(index=items, columns=[f"Loc{i}" for i in range(1,17)], dtype=float)
for item in items:
    s = storage_spaces.loc[item]
    for loc in range(1,17):
        total = 0.0
        for io in ['IO1','IO2','IO3']:
            f = freq.loc[item, io]
            c = unit_cost.loc[item, io]
            d = dist_df.loc[io, f"Loc{loc}"]
            total += (f * c * d)
        total /= s  # Normalize by storage spaces for the item
        weight.loc[item, f"Loc{loc}"] = total

weight_display = weight.copy()

print(weight.T)

                 A       B       C       D
Loc1   1876.666667  1020.0  1830.0  2907.5
Loc2   1471.666667   876.0  1308.0  2470.0
Loc3   1513.333333   996.0  1360.5  2650.0
Loc4   2001.666667  1380.0  1987.5  3447.5
Loc5   1681.666667   996.0  1968.0  2470.0
Loc6   1276.666667   852.0  1446.0  2032.5
Loc7   1318.333333   972.0  1498.5  2212.5
Loc8   1806.666667  1356.0  2125.5  3010.0
Loc9   1528.333333  1092.0  2158.5  2212.5
Loc10  1123.333333   948.0  1636.5  1775.0
Loc11  1165.000000  1068.0  1689.0  1955.0
Loc12  1653.333333  1452.0  2316.0  2752.5
Loc13  1416.666667  1308.0  2401.5  2135.0
Loc14  1011.666667  1164.0  1879.5  1697.5
Loc15  1053.333333  1284.0  1932.0  1877.5
Loc16  1541.666667  1668.0  2559.0  2675.0


### Mathematical Model (SLAP)

In [ ]:
# MIP leading to optimal product-location assignment
# If pulp not used, try scipy Hungarian on expanded copies

assignment = {}
obj_value = None
req = storage_spaces.to_dict()


expanded_items = []
for item in items:
    for k in range(req[item]):
        expanded_items.append(item)
# cost matrix: rows=expanded_items (16), cols=locations(16)
cost_mat = np.zeros((16,16))
for i,item in enumerate(expanded_items):
    for j in range(16):
        cost_mat[i,j] = weight.loc[item, f"Loc{j+1}"]
row_ind, col_ind = linear_sum_assignment(cost_mat)
obj_value = cost_mat[row_ind, col_ind].sum()
assignment = {}
for r,c in zip(row_ind, col_ind):
    assignment[c+1] = expanded_items[r]


# Build assignment dataframe
assign_df = pd.DataFrame({
    'Location': [f"Loc{loc}" for loc in range(1,17)],
    'Coord_x': [coords[loc][0] for loc in range(1,17)],
    'Coord_y': [coords[loc][1] for loc in range(1,17)],
    'Assigned_Item': [assignment[loc] for loc in range(1,17)],
    'Weight': [weight.loc[assignment[loc], f"Loc{loc}"] for loc in range(1,17)]
})

print(f"Objective value (total travel cost): {obj_value:.2f}\n")
print(assign_df.to_string(index=False))

# Plot assignment
fig, ax = plt.subplots(figsize=(6,6))
for loc in range(1,17):
    x0, y0 = coords[loc]
    rect = plt.Rectangle((x0-0.5, y0-0.5), 1, 1, fill=False, linewidth=1)
    ax.add_patch(rect)
for io, c in io_coords.items():
    ax.scatter(c[0], c[1], s=150, edgecolors='black', marker='o')
    ax.text(c[0], c[1]+0.15, io, ha='center', va='bottom')
for loc in range(1,17):
    x0, y0 = coords[loc]
    item = assignment[loc]
    ax.scatter(x0, y0, s=500, marker='o')
    ax.text(x0, y0, f"{loc}\n{item}", ha='center', va='center', color='white', fontsize=8, fontweight='bold')
ax.set_xlim(0,5); ax.set_ylim(0,5)
ax.set_xticks([]); ax.set_yticks([])
ax.set_title(f"Product-location assignment (Cost = {obj_value:.2f}) -- solver: {solver_used}")
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

assign_df.to_csv('/mnt/data/optimal_assignment.csv', index=False)
print("\nAssignment CSV saved to: sandbox:/mnt/data/optimal_assignment.csv")

Example optimal assignment (illustrative only):

```
Min. travel cost: 42,420
 ╔═════════════ I/O ═════╦═══════╗
 ║   A       A       A   ║   B   ║
 ╠═══════════════╦═══════╣       ║
 ║   D       D   ║   C   ║   B   ║
I/O              ║       ║       ║
 ║   D       D   ║   C   ║   B   ║
 ║               ╠═══════╝       ║
 ║   D       D   ║   B       B   ║
 ╚═════════════ I/O ═════════════╝
```

### 2. Random vs. Dedicated Policies

The answer comprises:

- Total storage for dedicated policy.
- Total storage for random policy.
- Reasoning behind the totals.